In [ ]:
refrst = '/mnt/disk1/jasp/a2autocls2023/imglsb_clip/L2A_T29SMC_20180115T112419_B02_10m.tif'

lulcshp = '/mnt/disk1/jasp/a2autocls2023/lulc_fmosm/lulc_lisboa_clc.shp'
lulccol = 'cls'

out_folder = '/mnt/disk1/jasp/a2autocls2023/trainres'

In [ ]:
import os
import pandas as pd

from glass.pys.oss       import mkdir, cpu_cores, fprop
from glass.dtt.rst.split import nrsts_fm_rst
from glass.dtt.split     import split_shp_by_attr
from glass.pd.split import df_split
from glass.wenv.grs import run_grass
from glass.gp.gen import dissolve

### Split study area in several parts

In [ ]:
reffolder = mkdir(os.path.join(out_folder, 'ref_rasters'), overwrite=True)

refparts = nrsts_fm_rst(refrst, 500, 500, reffolder, 'refrst')

dfareas = pd.DataFrame(refparts, columns=['study_areas'])

### Create one shapefile for each lulc class

In [ ]:
lulc_folder = mkdir(os.path.join(out_folder, 'lulc'), overwrite=True)

lulcs = split_shp_by_attr(
    lulcshp, lulccol, lulc_folder,
    outname='l', valinname=True
)

In [ ]:
# CPU Cores
n_cpu = cpu_cores()

# Split data by CPU
dfs = df_split(dfareas, n_cpu)

In [ ]:
def fishnet_to_train(tid, ref, _lulcs, output, ws):
    ifld = mkdir(os.path.join(ws, f'ires_{tid}'), overwrite=True)
    
    # Start GRASS GIS Session
    loc = f'loc_{str(tid)}'
    gb = run_grass(ws, location=loc, srs=ref)
    
    import grass.script.setup as gsetup
    
    gsetup.init(gb, ws, loc, 'PERMANENT')
    
    from glass.smp.fish import grass_fishnet
    from glass.it.shp import shp_to_grs, grs_to_shp
    from glass.gp.ovl.grs import grsintersection
    from glass.prop.feat import feat_count
    from glass.tbl.attr import geomattr_to_db
    from glass.tbl.joins import join_table
    
    # Create Fishnet
    fishnet = grass_fishnet(f'fishnet_{str(tid)}', ascmd=True)
    
    # Intersect with classes
    for k in _lulcs:
        # Import classes into GRASS GIS
        grslulc = shp_to_grs(_lulcs[k], f'l_{k}', filterByReg=True)
        
        if not feat_count(grslulc, gisApi='grass', work=ws, loc=loc):
            continue
        
        # Intersection
        igrs = grsintersection(
            fishnet, grslulc, f"i_{grslulc}",
            cmd=True
        )
        geomattr_to_db(igrs, f'c{k}', "area", "boundary", unit="meters")
        
        # Export Intersection result
        ishp = grs_to_shp(igrs, os.path.join(ifld, f"{igrs}.shp"), 'area')
        
        # Dissolve
        idiss = dissolve(
            ishp, os.path.join(ifld, f"{igrs}_diss.shp"),
            'a_cat',
            statistics={f'c{k}' : 'SUM'}
        )
        
        # Dissolve result to GRASS
        grsdiss = shp_to_grs(idiss, fprop(idiss, 'fn'))
        
        # Join with original Fishnet
        join_table(fishnet, grsdiss, 'cat', 'a_cat')
    
    # Export fishnet
    grs_to_shp(fishnet, output, 'area')

In [ ]:
ref_raster = dfs[0].study_areas.tolist()[0]

In [ ]:
fishnet_to_train(
    1, ref_raster, lulcs,
    os.path.join(out_folder, 'fishnet_00.shp'),
    out_folder
)

In [ ]:
ref_raster